In [2]:
import hashlib
import json


class Block:
    def __init__(self, index, data, timestamp, previous_hash, nonce=0):
        self.index = index
        self.data = data
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = nonce

    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return hashlib.sha256(block_string.encode()).hexdigest()

    def __repr__(self):
        return 'Block: {}'.format(self.__dict__)



In [6]:
import time
from timeit import default_timer as timer
from datetime import timedelta


class Blockchain:
    def __init__(self):
        self.new_data = []
        self.chain = []
        self.difficulty = 1
        self.__create_genesis_block()

    def __create_genesis_block(self):
        genesis_block = Block(0, [], time.time(), 0)
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)

    def add_new_data(self, data):
        self.new_data.append(data)

    def mine(self):
        if not self.new_data:
            return False
        last_block = self.last_block
        new_block = Block(index=last_block.index + 1,
                          data=self.new_data,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)
        proof = self.__proof_of_work(new_block)
        self.__add_block(new_block, proof)
        self.new_data = []
        return new_block.index

    @property
    def last_block(self):
        return self.chain[-1]
    
    def __proof_of_work(self, block):
        start = timer()
        block.nonce = 0
        computed_hash = block.compute_hash()
        print('Mining block {}'.format(block.index))
        print('Difficulty: {}'.format(self.difficulty))
        while not computed_hash.startswith('0' * self.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        print('Computed hash: {}'.format(computed_hash))
        print('Nonce: {}'.format(block.nonce))
        end = timer()
        print('Elapsed time: {}'.format(timedelta(seconds=end - start)))
        return computed_hash

    def __is_valid_proof(self, block, block_hash):
        return block_hash.startswith('0' * self.difficulty) and block_hash == block.compute_hash()

    def __add_block(self, block, proof):
        previous_hash = self.last_block.hash
        if previous_hash != block.previous_hash:
            return False
        if not self.__is_valid_proof(block, proof):
            return False
        block.hash = proof
        self.chain.append(block)
        return True

In [9]:
blockchain = Blockchain()
print(blockchain.chain)
for i in range(1, 10):
    data = {
        'name': 'Mining Block {}'.format(i),
    }
    blockchain.add_new_data(data)
    blockchain.mine()
print(blockchain.chain)



[Block: {'index': 0, 'data': [], 'timestamp': 1666659663.478511, 'previous_hash': 0, 'nonce': 0, 'hash': '892717a6c7634310a223e34224862cb534520167dd3b6d59aa575fe0a8086a2e'}]
Mining block 1
Difficulty: 1
Computed hash: 00a64a7e4339ae4b2a4f0fc45123bbe005ee149422d1093ef6388fc4e4e4eac5
Nonce: 9
Elapsed time: 0:00:00.000291
Mining block 2
Difficulty: 1
Computed hash: 0fbb60563a953769fa68d746ef32d8ed1577f7c4d5de1139021c44b5efb64200
Nonce: 8
Elapsed time: 0:00:00.000369
Mining block 3
Difficulty: 1
Computed hash: 08fafb8699c100885af222f3ac1ce4907b19294f15da5d0abe8ebaf02ac497e3
Nonce: 3
Elapsed time: 0:00:00.000214
Mining block 4
Difficulty: 1
Computed hash: 0761954a10f017b2aed9a675156872892c9f9e8f7de2b22799cfbdc87a6cd4fb
Nonce: 22
Elapsed time: 0:00:00.001568
Mining block 5
Difficulty: 1
Computed hash: 0112d8e76d04f3d98e7e75b10e9af10f5cf7b458948eabab8fd4d6bc36a7d5cb
Nonce: 25
Elapsed time: 0:00:00.000492
Mining block 6
Difficulty: 1
Computed hash: 0110b712380e6237aaa344ffd73627f20863c2ecbe24a

In [8]:
print()